# Домашняя работа по теме "Машинное обучение ранжированию"

В этом ДЗ мы:
- научимся работать со стандартным датасетом для машинного обучения ранжированию [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- попробуем применить на практике все то, чему мы научились на семинаре

## Как будет происходить сдача ДЗ

Вам надо:
- форкнуть эту репу
- создать бранч в котором вы дальше будете работать
- реализовать класс Model в этом ноутбуке
- убедиться, что ваша реализация выбивает NDCG@10 выше бейзлайна (см. ниже)
- запушить ваш бранч и поставить Pull Request
- в комментарии написать какой скор вы выбили

В таком случае мы (организаторы):

- счекаутим вашу бранчу
- проверим что ваша реализация действительно выбивает заявленный скор

Предполагается, что и вы, и мы работаем в виртаульном окружении как в семинаре про машинное обучение ранжированию: seminars/7-learning-to-rank/requirements.txt(подробнее про работу с виртуальными окружениями README в корне этой репы).

Оценка:
- За выбитый скор больше **0.507** назначаем **5** баллов, за скор больше (или равно) **0.510** назначаем максимальный балл -- 10 баллов
- Тот из участников кто выбъет самый высокий скор получит еще +10 баллов

При сдаче кода важно помнить о том, что:
- В коде не должно быть захардкоженных с потолка взятых гиперпараметров (таких как число деревьев, learning rate и т.п.) -- обязательно должен быть представлен код который их подбирает!
- Решение должно быть стабильно от запуска к запуску (на CPU) т.е. все seed'ы для генераторов случайных чисел должны быть фиксированы
- Мы (организаторы) будем запускать код на CPU поэтому, даже если вы использовали для подбора параметров GPU, финальный скор надо репортить на CPU

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [1]:
import pathlib
from timeit import default_timer as timer

import numpy as np
import pandas as pd

import catboost
from catboost import datasets, utils

import copy

Загрузим датасет MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Строго говоря, он состоит их 2х частей:

- основной датасет MSLR-WEB30K -- он содержит более 30 тыс. запросов
- "маленький" датасет MSLR-WEB10K, который содержит только 10 тыс. запросов и является случайным сэмплом датасета MSLR-WEB30K

в этом ДЗ мы будем работать с MSLR-WEB10K, т.к. полная версия датасета может просто не поместиться у нас в RAM (и, тем более, в память видеокарты если мы учимся на GPU)

Будем считать, что мы самостоятельно скачали датасет MSLR-WEB10K с официального сайта, поместили его в папку КОРЕНЬ-ЭТОЙ-РЕПЫ/data/mslr-web10k и раззиповали.

В результате у нас должна получиться следующая структура папок:

In [2]:
!ls ../../data

Fold1  Fold2  Fold3  Fold4  Fold5


In [4]:
!ls -lh ../../data/

итого 20K
drwxrwxr-x 2 james james 4,0K апр 28  2010 Fold1
drwxrwxr-x 2 james james 4,0K апр 28  2010 Fold2
drwxrwxr-x 2 james james 4,0K апр 28  2010 Fold3
drwxrwxr-x 2 james james 4,0K апр 28  2010 Fold4
drwxrwxr-x 2 james james 4,0K апр 28  2010 Fold5


Заметим, что датасет довольно большой, в распакованном виде он весит 7.7 GB.

Датасет состоит из нескольких фолдов, которые по сути представляют из себя разные разбиения одних и тех же данных на обучающее, валидационное и тестовые множеста.

Дальше мы будем использовать только первый фолд: Fold1.

Заглянем внутрь:

In [5]:
!ls -lh ../../data/Fold1

итого 4,1G
-rw-rw-r-- 1 james james 832M апр 30  2010 test.txt
-rw-rw-r-- 1 james james 2,5G апр 30  2010 train.txt
-rw-rw-r-- 1 james james 825M апр 30  2010 vali.txt


Видим, что у нас 3 файла с говорящими названиями, соответсвующими сплитам нашего датасета.

Посмотрим на содержимое одного из файлов:

In [6]:
!head -n 1 ../../data/Fold1/train.txt

2 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 16:6.931275 17:22.076928 18:19.673353 19:22.255383 20:6.926551 21:3 22:3 23:0 24:0 25:6 26:1 27:1 28:0 29:0 30:2 31:1 32:1 33:0 34:0 35:2 36:1 37:1 38:0 39:0 40:2 41:0 42:0 43:0 44:0 45:0 46:0.019231 47:0.75000 48:0 49:0 50:0.035928 51:0.00641 52:0.25000 53:0 54:0 55:0.011976 56:0.00641 57:0.25000 58:0 59:0 60:0.011976 61:0.00641 62:0.25000 63:0 64:0 65:0.011976 66:0 67:0 68:0 69:0 70:0 71:6.931275 72:22.076928 73:0 74:0 75:13.853103 76:1.152128 77:5.99246 78:0 79:0 80:2.297197 81:3.078917 82:8.517343 83:0 84:0 85:6.156595 86:2.310425 87:7.358976 88:0 89:0 90:4.617701 91:0.694726 92:1.084169 93:0 94:0 95:2.78795 96:1 97:1 98:0 99:0 100:1 101:1 102:1 103:0 104:0 105:1 106:12.941469 107:20.59276 108:0 109:0 110:16.766961 111:-18.567793 112:-7.760072 113:-20.838749 114:-25.436074 115:-14.518523 116:-21.710022 117:-21.339609 118:-24.497864 119:-27.690319 120:-20.203779 121:-15.449379 122:-4.474452 123:-23.634899 

Видим, что данные лежат в уже знакомом нам по семинару формате:

- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

В файле qid и все-фичи кодируются в формате КЛЮЧ:ЗНАЧЕНИЕ, напр. 130:116 -- тут 130 это номер фичи, а 116 -- ее значение.

Такой формат в мире машинного обучения часто называют svm light формат (в честь когда-то популярной библиотеки SVM-Light)

Напишем немного вспомогательного кода для загрузки этого датасета:

In [2]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns
    
def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1
                                  
                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")
    
                # Parse every field
                x = np.zeros(exp_num_fields, dtype=np.float32)
                label = int(fields[0])
                x[0] = label
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x[1] = qid
                for i, field in enumerate(fields[2:]):
                    _, feature_str = _parse_field(field)
                    x[i+2] = float(feature_str)
    
                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(f"Loaded {num_lines} lines...")
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e
    
    # To pandas
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

И теперь загрузим датасет:

In [3]:
fold_dir = pathlib.Path("../../data/Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"))
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"))
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"))
print(f"Dataset loaded from fold_dir {fold_dir}")

Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded 250000 lines...
Loaded 300000 lines...
Loaded 350000 lines...
Loaded 400000 lines...
Loaded 450000 lines...
Loaded 500000 lines...
Loaded 550000 lines...
Loaded 600000 lines...
Loaded 650000 lines...
Loaded 700000 lines...
Loaded 750000 lines...
Loaded 800000 lines...
Loaded 850000 lines...
Loaded 900000 lines...
Loaded 950000 lines...
Loaded 1000000 lines...
Loaded 1050000 lines...
Loaded 1100000 lines...
Loaded 1150000 lines...
Loaded 1200000 lines...
Loaded 1250000 lines...
Loaded 1300000 lines...
Loaded 1350000 lines...
Loaded 1400000 lines...
Loaded 1450000 lines...
Loaded 1500000 lines...
Loaded 1550000 lines...
Loaded 1600000 lines...
Loaded 1650000 lines...
Loaded 1700000 lines...
Loaded 1750000 lines...
Loaded 1800000 lines...
Loaded 1850000 lines...
Loaded 1900000 lines...
Loaded 1950000 lines...
Loaded 2000000 lines...
Loaded 2050000 lines...
Loaded 2100000 lines...
Loaded 21500

Посмотрим на данные:

In [4]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0  1.0        3.0        3.0        0.0        0.0        3.0   
1    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
2    0.0  1.0        3.0        0.0        2.0        0.0        3.0   
3    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
4    1.0  1.0        3.0        0.0        3.0        0.0        3.0   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...         62.0   11089534.0   
1        1.0        0.0   1.000000  ...         54.0   11089534.0   
2        1.0        0.0   0.666667  ...         45.0          3.0   
3        1.0        0.0   1.000000  ...         56.0   11089534.0   
4        1.0        0.0   1.000000  ...         64.0          5.0   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0          2.0        116.0      64034.0         13.0          3.0   
1          2

Т.е. теперь мы видим что данные доступны в точно таком же виде, как это было в семинаре.

Проведем небольшой EDA.

Всего у нас 723 тыс. документов в трейне:

In [5]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2270296 entries, 0 to 2270295
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 1.2 GB
None


235 тыс. документов в валидации:

In [6]:
print(df_valid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747218 entries, 0 to 747217
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 393.4 MB
None


И 241 тыс. документов в тесте:

In [7]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753611 entries, 0 to 753610
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 396.7 MB
None


Сколько у нас запросов?

In [8]:
num_queries_train = df_train['qid'].nunique()
num_queries_valid = df_valid['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train, {num_queries_valid} valid and {num_queries_test} test queries")

Got 18919 train, 6306 valid and 6306 test queries


In [34]:
def to_catboost_dataset(df):
    check_list = df['qid'].value_counts()

    for indx, values in check_list.items():
        if values > 1023:
            print(indx, values)

            indxs = df[df['qid'] == indx].index
            to_delete = values - 1023

            for i in range(to_delete):
                print(i)
                df = df.drop(indxs[i])
        else:
            break

    y = df["label"].to_numpy()  # Label: [0-4]
    q = df["qid"].to_numpy().astype("uint32")  # Query Id
    X = df.drop(columns=["label", "qid"]).to_numpy()  # 136 features
    return (X, y, q)


X_train, y_train, q_train = to_catboost_dataset(df_train)
X_valid, y_valid, q_valid = to_catboost_dataset(df_valid)
X_test, y_test, q_test = to_catboost_dataset(df_test)

21563.0 1251
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
14751.0 1031
0
1
2
3
4
5
6
7


In [35]:
pool_train = catboost.Pool(data=X_train, label=y_train, group_id=q_train)
pool_valid = catboost.Pool(data=X_valid, label=y_valid, group_id=q_valid)
pool_test = catboost.Pool(data=X_test, label=y_test, group_id=q_test)

In [36]:
EVAL_METRIC = "NDCG:top=10;type=Exp"

In [51]:
DEFAULT_PARAMS = {
    "iterations": 8000,  # maximum possible number of trees
    "early_stopping_rounds": 100,  # stop if metric does not improve for N rounds
    "eval_metric": EVAL_METRIC,  # # metric used for early stopping
    "random_seed": 22,
    "verbose": 10,
    'task_type': 'GPU',
}

## Обучаем модель

Теперь можно приступить непосредственно к обучению модели. 

Объявим класс модели, который надо будем заимлементить в этом ДЗ:

In [52]:
class Model:
    def __init__(self):
        params = copy.deepcopy(DEFAULT_PARAMS)

        loss_function = "YetiRank"
        # Temporary directory that is used by catboost to store additional information
        catboost_info_dir = f"/tmp/catboost_info.{loss_function.lower()}"

        params.update(
            {
                "loss_function": loss_function,
                "train_dir": str(catboost_info_dir),
            }
        )

        self.model = catboost.CatBoost(params)

    def fit(self, pool_train, pool_valid):
        start = timer()
        self.model.fit(pool_train, eval_set=pool_valid, use_best_model=True)
        elapsed = timer() - start

        print(f"Model fit: elapsed = {elapsed:.3f} num_trees = {self.model.tree_count_}")

    def predict(self, pool_test):
        return self.model.predict(pool_test)

Создадим и применим модель:

In [53]:
# Create model
model = Model()

# Fit
start = timer()
model.fit(pool_train, pool_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

# Predict
y_hat_test = model.predict(pool_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2792630	best: 0.2792630 (0)	total: 70.1ms	remaining: 9m 20s
10:	test: 0.3656408	best: 0.3656408 (10)	total: 494ms	remaining: 5m 58s
20:	test: 0.3738353	best: 0.3738353 (20)	total: 917ms	remaining: 5m 48s
30:	test: 0.3922902	best: 0.3923130 (29)	total: 1.34s	remaining: 5m 44s
40:	test: 0.4015678	best: 0.4015678 (40)	total: 1.76s	remaining: 5m 42s
50:	test: 0.4056327	best: 0.4056485 (48)	total: 2.19s	remaining: 5m 40s
60:	test: 0.4090830	best: 0.4090830 (60)	total: 2.62s	remaining: 5m 40s
70:	test: 0.4128120	best: 0.4129138 (69)	total: 3.04s	remaining: 5m 39s
80:	test: 0.4160283	best: 0.4160566 (77)	total: 3.46s	remaining: 5m 38s
90:	test: 0.4204706	best: 0.4204706 (90)	total: 3.89s	remaining: 5m 37s
100:	test: 0.4249529	best: 0.4250321 (98)	total: 4.32s	remaining: 5m 37s
110:	test: 0.4273936	best: 0.4273936 (110)	total: 4.74s	remaining: 5m 37s
120:	test: 0.4294234	best: 0.4294234 (120)	total: 5.17s	remaining: 5m 36s
130:	test: 0.4324591	best: 0.4324591 (130)	total: 5.59s	rema

Теперь, имея предикты, можно посчитать метрики качества:

In [55]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

# Get test targets and groups
y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.506


Ожидаем, что ваша модель покажет результаты выше бейзлайна!